In [ ]:
!pip install langgraph

In [ ]:
!pip install langchain-google-genai google-generativeai

In [ ]:
from typing import TypedDict,Optional
from langgraph.graph import StateGraph
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.schema import HumanMessage

In [ ]:
class GraphState(TypedDict): #sharedstate
    resume:Optional[str]
    qualified:Optional[bool]
    message:Optional[str]

In [ ]:
import os
os.environ["GOOGLE_API_KEY"]="your_api_key"
llm=ChatGoogleGenerativeAI(model='gemini-1.5-flash',temperature=0,api_key=os.environ["GOOGLE_API_KEY"])

In [ ]:
def load_resume(state):
    resume = """your resume content """
    return {"resume":resume}

In [ ]:
def check_qualification(state):
    resume=state["resume"]
    prompt=f"""Based on this resume, is the candidate qualified for an Software development job?
    Answer only YES or NO.\n\n{resume}"""
    response=llm.invoke([HumanMessage(content=prompt)])
    qualified="YES" in response.content.upper()
    return {"resume":resume,"qualified":qualified}

def write_positive(state):
    resume=state["resume"]
    prompt=f"""Write a positive reply for this
    qualified candidate:\n{resume}"""
    response=llm.invoke([HumanMessage(content=prompt)])
    return {"message":response.content}

def write_rejection(state):
    resume=state["resume"]
    prompt=f"Politely reject this candidate:\n{resume}"
    response=llm.invoke([HumanMessage(content=prompt)])
    return {"message":response.content}

def route_decision(state):
    return "positive feedback" if state["qualified"] else "rejection message"

In [ ]:
graph=StateGraph(GraphState)
graph.add_node("load resume",load_resume)
graph.add_node("qualification",check_qualification)
graph.add_node("positive feedback",write_positive)
graph.add_node("negative message",write_rejection)

graph.set_entry_point("load resume")
graph.add_edge("load resume","qualification")
graph.add_conditional_edges("qualification",route_decision)
graph.set_finish_point("positive feedback")
graph.set_finish_point("negative message")

In [ ]:
gr=graph.compile()
output=gr.invoke({})

print("Resume:")
print(output["resume"])
print("\n Result:","Qualified" if output["qualified"] else "Not qualified")
print("\n Final Message:")
print(output["message"])